In [1]:
from zipfile import ZipFile
import numpy as np
import pandas as pd
import zipfile
import re
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure



zf = zipfile.ZipFile('Data/Data_incidents.csv.zip') 
data = pd.read_csv('Data/Data_incidents.csv.zip').reset_index()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Extract mass shooting incidents

In [2]:
data = data.drop(columns=['index','Unnamed: 0'])

In [3]:
mass_shootings = data[data['Mass Shooting']==True]

In [4]:
len(mass_shootings)

1363

## Incidents over time

In [5]:
ms_time = mass_shootings.groupby('Date')['incident_id'].nunique()
ms_time

Date
2014-01-01    1
2014-01-03    1
2014-01-12    3
2014-01-13    1
2014-01-14    1
             ..
2018-03-17    2
2018-03-21    1
2018-03-24    1
2018-03-29    2
2018-03-31    1
Name: incident_id, Length: 808, dtype: int64

In [6]:
ms_time = ms_time.to_frame()
ms_time.reset_index(level=0, inplace=True)

In [7]:
ms_time

,Date,incident_id
0,2014-01-01,1
1,2014-01-03,1
2,2014-01-12,3
3,2014-01-13,1
4,2014-01-14,1
...,...,...
803,2018-03-17,2
804,2018-03-21,1
805,2018-03-24,1
806,2018-03-29,2


In [8]:
#check sum
ms_time['incident_id'].sum()

1363

In [9]:
# format date
ms_time['date'] = pd.to_datetime(ms_time.Date.str.strip(), format = '%Y-%m-%d')#
ms_time=ms_time.drop(columns = 'Date');

In [10]:
ms_time

,incident_id,date
0,1,2014-01-01
1,1,2014-01-03
2,3,2014-01-12
3,1,2014-01-13
4,1,2014-01-14
...,...,...
803,2,2018-03-17
804,1,2018-03-21
805,1,2018-03-24
806,2,2018-03-29


In [11]:
# Create new date columns
ms_time['Year'] = ms_time.date.apply(lambda x: x.year)
ms_time['Month'] = ms_time.date.apply(lambda x: x.month)
ms_time['Year_Month'] = ms_time.date.apply(lambda x: 100*x.year + x.month)

In [12]:
ms_time.head(2)

,incident_id,date,Year,Month,Year_Month
0,1,2014-01-01,2014,1,201401
1,1,2014-01-03,2014,1,201401


In [13]:
#ms_year = ms_time.groupby('Year').incident_id.sum()
ms_year_month = ms_time.groupby('Year_Month').incident_id.sum()

In [14]:
ms_year_month = ms_year_month.to_frame()
ms_year_month.reset_index(level=0, inplace=True)
ms_year_month.columns = ['Year_Month', 'ms_all']

In [15]:
inc_all = data.Date
inc_all = inc_all.to_frame()
inc_all['counts'] = 1

In [16]:
inc_all['date'] = pd.to_datetime(inc_all.Date.str.strip(), format = '%Y-%m-%d')#
inc_all= inc_all.drop(['Date'], axis=1);

In [17]:
inc_all['Year'] = inc_all.date.apply(lambda x: x.year)
inc_all['Year_Month'] = inc_all.date.apply(lambda x: 100*x.year + x.month)

In [18]:
all_YM = inc_all.groupby('Year_Month').counts.sum()

In [19]:
all_YM_pd = all_YM.to_frame()
all_YM_pd.reset_index(level=0, inplace=True)
all_YM_pd.columns = ['Year_Month', 'inc_all']

In [20]:
YM = pd.merge(all_YM_pd, ms_year_month, on='Year_Month')

In [21]:
len(YM)
YM.ms_all.sum()

1363

In [22]:
mass_shootings['victims'] = mass_shootings.n_killed + mass_shootings.n_injured


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
max_vic = mass_shootings.victims.max()
mass_shootings[mass_shootings['victims']==max_vic]

,incident_id,state,city_or_county,n_killed,n_injured,incident_characteristics,latitude,location_description,longitude,participant_age,...,Child injured,Gun buy back action,Terrorism Involvement,Officer Involved Shooting - Accidental discharge - no injury required,Mistaken ID,Officer Involved Shooting - subject/suspect/perpetrator suicide by cop,Ghost gun,Political Violence,NAV,victims
127677,577157,Florida,Orlando,50,53,"Shot - Wounded/Injured||Shot - Dead (murder, a...",28.5195,Pulse,-81.3767,0::34||1::23||2::20||3::22||4::36||5::22||6::2...,...,False,False,True,False,False,False,False,False,False,103


In [51]:
mass_shootings.groupby('victims')['incident_id'].nunique()

victims
4      759
5      337
6      111
7       71
8       33
9       16
10       7
11       3
12       7
13       1
14       1
15       3
16       1
17       2
18       2
19       1
20       1
21       1
25       1
27       1
34       1
35       1
47       1
103      1
Name: incident_id, dtype: int64

In [53]:
for col in mass_shootings.columns:
    print(col)

incident_id
state
city_or_county
n_killed
n_injured
incident_characteristics
latitude
location_description
longitude
participant_age
participant_age_group
participant_gender
participant_name
participant_relationship
participant_status
participant_type
Date
Year
Shot - Wounded/Injured
Mass Shooting
Possession
Possession of gun by felon or prohibited person
Shot - Dead
Gang involvement
Shots Fired - No Injuries
Bar/club incident - in or around establishment
Officer Involved Incident
Officer Involved Shooting - subject/suspect/perpetrator killed
Drug involvement
Kidnapping/abductions/hostage
Under the influence of alcohol or drugs
Suicide
Murder/Suicide
Attempted Murder/Suicide
Domestic Violence
Home Invasion
Home Invasion - Resident killed
Armed robbery with injury/death and/or evidence of DGU found
Drive-by
Officer Involved Shooting - Officer shot
Officer Involved Shooting - Officer killed
Officer Involved Shooting - subject/suspect/perpetrator shot
Officer Involved Shooting - subject/s

## TABLE OF CHARACTERISTICS

In [52]:
df_incidents = mass_shootings.iloc[:, list(range(18,128))]
incidents = []

for col in df_incidents.columns: 
    incidents.append(col)

In [40]:
dataframes_gender = []

for row in mass_shootings['participant_gender'].fillna("0:,")[:len(mass_shootings)]:
    participants = np.array([re.split(r'::?', p) for p in re.split(r'\|\|?', row)]).T
    #print('participants: ', participants)

    data = dict(zip(*participants))
    #print('data: ', data)

    dataframe = pd.Series(data).to_frame().T
    #display(dataframe)

    dataframes_gender.append(dataframe)

gender = pd.concat(dataframes_gender).reset_index().drop('index', 1).fillna('')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [42]:
dataframes_age = []

for row in mass_shootings['participant_age'].fillna("0:,")[:len(mass_shootings)]:
    participants = np.array([re.split(r'::?', p) for p in re.split(r'\|\|?', row)]).T
    #print('participants: ', participants)

    data = dict(zip(*participants))
    #print('data: ', data)

    dataframe = pd.Series(data).to_frame().T
    #display(dataframe)

    dataframes_age.append(dataframe)

age = pd.concat(dataframes_age).reset_index().drop('index', 1).fillna('')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [43]:
dataframes_status = []

for row in mass_shootings['participant_status'].fillna("0:,")[:len(mass_shootings)]:
    participants = np.array([re.split(r'::?', p) for p in re.split(r'\|\|?', row)]).T
    #print('participants: ', participants)

    data = dict(zip(*participants))
    #print('data: ', data)

    dataframe = pd.Series(data).to_frame().T
    #display(dataframe)

    dataframes_status.append(dataframe)

status = pd.concat(dataframes_status).reset_index().drop('index', 1).fillna('')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [44]:
dataframes_type = []

for row in mass_shootings['participant_type'].fillna("0:,")[:len(mass_shootings)]:
    participants = np.array([re.split(r'::?', p) for p in re.split(r'\|\|?', row)]).T
    #print('participants: ', participants)

    data = dict(zip(*participants))
    #print('data: ', data)

    dataframe = pd.Series(data).to_frame().T
    #display(dataframe)

    dataframes_type.append(dataframe)

_type = pd.concat(dataframes_type).reset_index().drop('index', 1).fillna('')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


### Gender of victimes and suspects

In [55]:
combined_gender = gender.astype(str).add(',').add(_type.astype(str)) #.add(', ').add(status.astype(str))
com = combined_gender.fillna(',')

com['Male, Victim'] = com.apply(lambda row: sum(row[0:103]=='Male,Victim') ,axis=1)
com['Female, Victim'] = com.apply(lambda row: sum(row[0:103]=='Female,Victim') ,axis=1)
com['Male, Suspect'] = com.apply(lambda row: sum(row[0:103]=='Male,Subject-Suspect') ,axis=1)
com['Female, Suspect'] = com.apply(lambda row: sum(row[0:103]=='Female,Subject-Suspect') ,axis=1)
com['Victim1'] = com.apply(lambda row: sum(row[0:103]==',Victim') ,axis=1)
com['Victim2'] = com.apply(lambda row: sum(row[0:103]==',,Victim') ,axis=1)
com['Subject'] = com.apply(lambda row: sum(row[0:103]==',Subject-Suspect') ,axis=1)

com[['Male, Victim','Female, Victim','Male, Suspect','Female, Suspect','Victim1','Victim2','Subject']].sum()

Male, Victim       3863
Female, Victim     1484
Male, Suspect      1086
Female, Suspect      33
Victim1            1386
Victim2             109
Subject             107
dtype: int64

### Status of victimes and suspects

In [56]:
combined_status = status.astype(str).add(',').add(_type.astype(str)) #.add(', ').add(status.astype(str))
com_s = combined_status.fillna(',')

com_s['Killed, Victim'] = com.apply(lambda row: sum(row[0:103]=='Killed,Victim') ,axis=1)
com_s['Injured, Victim'] = com.apply(lambda row: sum(row[0:103]=='Injured,Victim') ,axis=1)
com_s['Unharmed/Arrested, Victim'] = com.apply(lambda row: sum(row[0:103]=='Unharmed, Arrested,Victim') ,axis=1)
com_s['Killed, Suspect'] = com.apply(lambda row: sum(row[0:103]=='Killed,Subject-Suspect') ,axis=1)
com_s['Injured, Suspect'] = com.apply(lambda row: sum(row[0:103]=='Injured,Subject-Suspect') ,axis=1)
com_s['Unharmed/Arrested, Suspect'] = com.apply(lambda row: sum(row[0:103]=='Unharmed, Arrested,Subject-Suspect') ,axis=1)

com_s[['Killed, Victim', 'Injured, Victim','Unharmed/Arrested, Victim','Killed, Suspect','Injured, Suspect','Unharmed/Arrested, Suspect']].sum()

Killed, Victim                0
Injured, Victim               0
Unharmed/Arrested, Victim     0
Killed, Suspect               0
Injured, Suspect              0
Unharmed/Arrested, Suspect    0
dtype: int64

## Combined gender and status

In [66]:
combined_gender_status = gender.astype(str).add(',').add(status.astype(str)) #.add(', ').add(status.astype(str))
com_gs = combined_gender_status.fillna(',')

com_gs['Male, Killed'] = com_gs.apply(lambda row: sum(row[0:103]=='Male,Killed') ,axis=1)
com_gs['Female, Killed'] = com_gs.apply(lambda row: sum(row[0:103]=='Female,Killed') ,axis=1)
com_gs['Male, Injured'] = com_gs.apply(lambda row: sum(row[0:103]=='Male,Injured') ,axis=1)
com_gs['Female, Injured'] = com_gs.apply(lambda row: sum(row[0:103]=='Female,Injured') ,axis=1)
#com_gs['Victim1'] = com_gs.apply(lambda row: sum(row[0:103]==',Victim') ,axis=1)
#com_gs['Victim2'] = com_gs.apply(lambda row: sum(row[0:103]==',,Victim') ,axis=1)
#com_gs['Subject'] = com_gs.apply(lambda row: sum(row[0:103]==',Subject-Suspect') ,axis=1)

com_gs[['Male, Killed','Female, Killed','Male, Injured','Female, Injured']].sum()

Male, Killed       1089
Female, Killed      417
Male, Injured      2850
Female, Injured    1062
dtype: int64

In [59]:
com_s.loc[2]

0                                                 Injured,Victim
1                                                 Injured,Victim
10                            Unharmed, Arrested,Subject-Suspect
100                                                            ,
101                                                            ,
                                             ...                
Injured, Victim                                                0
Unharmed/Arrested, Victim                                      0
Killed, Suspect                                                0
Injured, Suspect                                               0
Unharmed/Arrested, Suspect                                     0
Name: 2, Length: 109, dtype: object

In [68]:
##SKAL GENNEMSKUES!

#stacked = combined.stack()
#stacked_df = pd.DataFrame(stacked)
#combinations = stacked_df.reset_index()
#stacked.unique()

In [61]:
def separate(df):
    df=df.split("||")
    df=[(x.split("::")) for x in df]
    y = []
    for  i in range (0, len(df)):
        y.append(df[i][-1])
    return(y) 

In [62]:
# Counting males and females 
mass_shootings['participant_gender'] = mass_shootings['participant_gender'].fillna("0::Zero")
mass_shootings['gender'] = mass_shootings['participant_gender'].apply(lambda x: separate(x))
mass_shootings['Males'] = mass_shootings['gender'].apply(lambda x: x.count('Male'))
mass_shootings['Females'] = mass_shootings['gender'].apply(lambda x: x.count('Female'))

# Counting victims and subjects    
mass_shootings['participant_type'] = mass_shootings['participant_type'].fillna("0::Zero")
mass_shootings['type'] = mass_shootings['participant_type'].apply(lambda x: separate(x))
mass_shootings['Victims'] = mass_shootings['type'].apply(lambda x: x.count('Victim'))
mass_shootings['Subject'] = mass_shootings['type'].apply(lambda x: x.count('Subject-Suspect'))

In [63]:
print(mass_shootings['Victims'].sum())
print(mass_shootings['Subject'].sum())

6793
1210


In [64]:
char = pd.DataFrame(mass_shootings[incidents].sum())
char.columns = ['count']
char = char[char['count'] >= 50]

In [65]:
char.sort_values('count', ascending=False)

,count
victims,6968.0
Mass Shooting,1363.0
Shot - Wounded/Injured,1300.0
Shot - Dead,715.0
Bar/club incident - in or around establishment,230.0
Drive-by,216.0
Gang involvement,160.0
Possession,138.0
Domestic Violence,93.0
Institution/Group/Business,89.0
